In [ ]:


class Crystal_input_base:
    def __init__(self):
        # Initialize the object to empty values
        self._blocklist = ['_title', 
                           '_block_geom', 
                           '_block_opt', 
                           '_block_optbs',
                           '_block_freq', 
                           '_block_anharm', 
                           '_block_cphf', 
                           '_block_elastcon', 
                           '_block_eos', 
                           '_block_bs',
                           '_block_scf', 
                           '_block_dft',
                           '_block_d3',
                           '_block_gcp']
        for i in self._blocklist:
            if i is '_title':
                setattr(self, i, 'Generated by CRYSTALpytools\n')
            else:
                setattr(self, i, '')

        
    


    
    def supercel(self, sp_matrix):
        """
        Corresponds to 'SUPERCEL' keyword
        
        Args:
            sp_matrix (array | list): 3 \* 3 or ndimen \* ndimen int matrix
        """
        return self._sp_matrix(sp_matrix, 'SUPERCEL\n')
    
    def supercon(self, sp_matrix):
        """
        Corresponds to 'SUPERCON' keyword
        
        Args:
            sp_matrix (array | list): 3 \* 3 or ndimen \* ndimen int matrix
        """
        return self._sp_matrix(sp_matrix, 'SUPERCON\n')
    
    def scelconf(self, sp_matrix):
        """
        Corresponds to 'SCELCONF' keyword
        
        Args:
            sp_matrix (array | list): 3 \* 3 or ndimen \* ndimen int matrix
        """
        return self._sp_matrix(sp_matrix, 'SCELCONF\n')
    
    def scelphono(self, sp_matrix):
        """
        Corresponds to 'SCELPHONO' keyword
        
        Args:
            sp_matrix (array | list): 3 \* 3 or ndimen \* ndimen int matrix
        """
        return self._sp_matrix(sp_matrix, 'SCELPHONO\n')

    def _sp_matrix(self, sp_matrix, command):
        """
        The base method for 'SUPCELL', 'SUPERCON', 'SCELCONF' and 'SCELPHONO'.
        
        Args:
            sp_matrix (array)
            command (str): CRYSTAL keyword
        """
        import numpy as np
        
        if not hasattr(self, '_dimenprt'):
            raise Exception('Dimension of the system is not defined.')
        if self._dimenprt is 'MOLECULE':
            raise Exception('0D system does not support any supercell option.')

        dimension_list = {
            'CRYSTAL'  : 3,
            'SLAB'     : 2,
            'POLYMER'  : 1,
            'HELIX'    : 1,
            'EXTERNAL' : np.size(sp_matrix),
            'DLVINPUT' : np.size(sp_matrix)
        }
        sp_matrix = np.array(sp_matrix, dtype=int)[:dimension_list[self._dimenprt], 
                                                   :dimension_list[self._dimenprt]]
        self._scelprt = command
        for i in sp_matrix:
            for j in i:
                self._scelprt += '-3i' % j
            self._scelprt += '\n'
        return
##############################################
    


In [ ]:
from typing import List
    

class Crystal_inputBASE:
    """
    The base class of Crystal_input class
    """
    
    
    

    
class BlockBASE():
    """
    The base class of 'block' objects
    """
    def __init__(self):
        self._block_bg = ''
        self._block_ed = ''
        self._block_data = ''
        self._block_dict = {}
        self._block_key = set(self._block_dict.keys())
        self._block_attr = set(self._block_dict.values())
    
    def __setattr__(self, attr, value):
        if attr in self._block_attr:
            setattr(self, attr, value)
        else:
            raise AttributeError('{} is not a valid attribute'.format(attr))
        return

    @staticmethod
    def _assign_keyword(key, value):
        """
        Transform value into string formats.
        
        Args:
            key (str): CRYSTAL keyword
            value (turple): Each element corresponds to a line of data.
        Returns:
            text (str): CRYSTAL input
        """
        text = key + '\n'
        if len(value) != 0:
            for v in value:
                text += str(v)
                text += '\n'
        return text
    
    def update_block(self):
        """
        Update the '_block_data' attribute
        """
        if len(self._block_bg) != 0 or len(self._block_ed) != 0:
            self._block_data = ''
            for attr in self._block_attr:
                try:
                    self._block_data += getattr(self, attr)
                except AttributeError:
                    continue
        return
     
    def analyze_text(self, text):
        """
        Analyze the input text and return to corresponding attributes
        """
        import warnings
            
        textline = text.strip().split('\n')
        value = ''
        attr = ''
        nline = 0
        while nline < len(textline):
            line = textline[nline]
            if line in self._block_key: # keyword line
                if len(attr) > 0: # Assign the previous keyword
                    setattr(self, attr, value)
                
                # update attribute
                attr = self._block_dict[line]
                value = line + '\n'
                if hasattr(self, attr):
                    warnings.warn(
                        "Attribute '{}' exists. The new entry will cover the old one".format(attr), 
                        stacklevel=2
                    )
            else: # value line
                value += line + '\n'

            nline +=1
        
        return

class BlockGeom(BlockBASE):
    """
    Geometry block object
    """
    def __init__(self):
        self._block_bg = 'Generated by CRYSTALpytools\n' # Set title as bg label
        self._block_ed = ''
        self._block_data = ''
        self._block_dict = {
            'CRYSTAL'   : '_basegeom',
            'SLAB'      : '_basegeom',
            'POLYMER'   : '_basegeom',
            'HELIX'     : '_basegeom',
            'MOLECULE'  : '_basegeom',
            'EXTERNAL'  : '_basegeom',
            'DLVINPUT'  : '_basegeom',
            'SUPERCEL'  : '_sp_matrix',
            'SUPERCELL' : '_sp_matrix',
            'SUPERCON'  : '_sp_matrix',
            'SCELCONF'  : '_sp_matrix',
            'SCELPHONO' : '_sp_matrix',
        }
    
    def title(self, title: str):
        self._block_bg = title
        
    @staticmethod
    def _set_structure(sg, latt, atom):
        geom = (sg, latt, len(atom))
        for a in atom:
            geom += (a,)
        return geom
        
    def crystal(self, sg, latt, atom, IFLAG=0, IFHR=0, IFSO=0, origin=[]):
        """
        Define 'CRYSTAL' structure
        
        Args:
            sg (int): Space group number
            latt (list): Minimal set of crystallographic cell parameters
            atom (list): Natom \* 4 list of conventional atomic number and 3D 
                fractional coordinates.
            IFLAG (int): See the manual
            IFHR (int): See the manual
            IFSO (int): See the manual
            origin (list): *IFSO > 1* See the manual
        """
        if IFSO <= 1:
            inp = ([IFLAG, IFHR, IFSO],)
        else:
            inp = ([IFLAG, IFHR, IFSO], origin,)
        
        inp += self._set_structure(sg, latt, atom)
        self._basegeom = super(BlockBASE, self)._assign_keyword('CRYSTAL', inp)
    
    def slab(self, sg, latt, atom):
        """
        Define 'SLAB' structure
        """
        self._basegeom = super(BlockBASE, self)._assign_keyword(
            'SLAB', self._set_structure(sg, latt, atom)
        )
    
    def polymer(self, sg, latt, atom):
        """
        Define 'POLYMER' structure
        """
        self._basegeom = super(BlockBASE, self)._assign_keyword(
            'POLYMER', self._set_structure(sg, latt, atom)
        )
    
    def helix(self, sg, latt, atom, N1=0, N2=0):
        """
        Define 'HELIX' structure
        
        Args:
            N1 (int): See the manual
            N2 (int): See the manual
        """
        inp = ([N1, N2],) + self._set_structure(sg, latt, atom)
        self._basegeom = super(BlockBASE, self)._assign_keyword('HELIX', inp)
        
    def molecule(self, sg, latt, atom):
        """
        Define 'MOLECULE' structure
        """
        self._basegeom = super(BlockBASE, self)._assign_keyword(
            'MOLECULE', self._set_structure(sg, latt, atom)
        )
    
    def external(self):
        """
        Define 'EXTERNAL' structure
        """
        self._basegeom = super(BlockBASE, self)._assign_keyword('EXTERNAL', ())
    
    def dlvinput(self):
        """
        Define 'DLVINPUT' structure
        """
        self._basegeom = super(BlockBASE, self)._assign_keyword('DLVINPUT', ())
    
    

class BlockOptgeom(BlockBASE):
    """
    OPTGEOM block object
    """
    def __init__(self):
        self._block_bg = 'OPTGEOM\n'
        self._block_ed = 'ENDOPT\n'
        self._block_data = ''
        self._block_dict = {
            'FULLOPTG' : '_opttype',
            'CELLONLY' : '_opttype',
            'INTREDUN' : '_opttype',
            'ITATOCEL' : '_opttype',
            'CVOLOPT'  : '_opttype',
            'HESSIDEN' : '_opthess',
            'HESSMOD1' : '_opthess',
            'HESSMOD2' : '_opthess',
            'HESSNUM'  : '_opthess',
            'TOLDEG'   : '_toldeg',
            'TOLDEX'   : '_toldex', 
            'TOLDEE'   : '_toldee',
            'MAXCYCLE' : '_maxcycle',
            'FRAGMENT' : '_fragment',
            'RESTART'  : '_restart',
            'FINALRUN' : '_finalrun',
            'EXTPRESS' : '_extpress',
        }
        self._block_key = set(self._block_dict.keys())
        self._block_attr = set(self._block_key.values())

    def fulloptg(self):
        self._opttype = super(BlockBASE, self)._assign_keyword('FULLOPTG', ())
    
    def cellonly(self):
        self._opttype = super(BlockBASE, self)._assign_keyword('CELLONLY', ())
    
    def intredun(self):
        self._opttype = super(BlockBASE, self)._assign_keyword('INTREDUN', ())
    
    def itatocel(self):
        self._opttype = super(BlockBASE, self)._assign_keyword('ITATOCEL', ())
    
    def cvolopt(self):
        self._opttype = super(BlockBASE, self)._assign_keyword('CVOLOPT', ())
    
    def hessiden(self):
        self._opthess = super(BlockBASE, self)._assign_keyword('HESSIDEN', ())
    
    def hessmod1(self):
        self._opthess = super(BlockBASE, self)._assign_keyword('HESSMOD1', ())
    
    def hessmod2(self):
        self._opthess = super(BlockBASE, self)._assign_keyword('HESSMOD2', ())
    
    def hessnum(self):
        self._opthess = super(BlockBASE, self)._assign_keyword('HESSNUM', ())

    def toldeg(self, TG):
        self._toldeg = super(BlockBASE, self)._assign_keyword('TOLDEG', (TG))

    def toldex(self, TX):
        self._toldex = super(BlockBASE, self)._assign_keyword('TOLDEX', (TX))

    def toldee(self, IG):
        self._toldee = super(BlockBASE, self)._assign_keyword('TOLDEE', (IG))
    
    def maxcycle(self, MAX):
        self._maxcycle = super(BlockBASE, self)._assign_keyword('MAXCYCLE', (MAX))
    
    def fragment(self, NL, LB):
        self._fragment = super(BlockBASE, self)._assign_keyword('FRAGMENT', (NL, LB))
    
    def restart(self):
        self._restart = super(BlockBASE, self)._assign_keyword('RESTART', ())
    
    def finalrun(self, ICODE):
        self._finalrun = super(BlockBASE, self)._assign_keyword('FINALRUN', (ICODE))
    
    def extpress(self, pres, unit='AU'):
        """
        .. Note::
            ``unit='GPa'`` is also allowed.
        """
        import re
        from CRYSTALpytools.units import GPa_to_pAU
        
        if re.match(r'^GPa$', unit, re.IGNORECASE):
            pres = pres * GPa_to_pAU
        self._extpress = super(BlockBASE, self)._assign_keyword('EXTPRESS', (pres))
    

In [11]:
class man:
    def __init__(self):
        pass
    def idnum(self, idnum: int):
        print(idnum)


In [13]:
a = man()
a.idnum(3.3333)

3.3333


In [4]:
import warnings

warnings.warn('test', stacklevel=2)

/work/e89/e89/hyz20ic/apps/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: UserWarning: test
  exec(code_obj, self.user_global_ns, self.user_ns)
